# Practica 3 - Web Scraping y Data Wrangling

### Mario Alberto Lizarraga Reyes


## Práctica 1

Usando requests recupere la página https://www.bloghemia.com/2019/05/noam-chomsky-michel-foucault-debate.html luego, usando beautifulsoup y pandas realizar los siguientes pasos con ella:


In [85]:
#%pip install requests beautifulsoup4 pandas openpyxl xlwt

import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import json
import re


In [ ]:

# URL del debate
url = "https://www.bloghemia.com/2019/05/noam-chomsky-michel-foucault-debate.html"

# Obtener el contenido de la página
response = requests.get(url)
response.encoding = 'utf-8'  # Asegurar la codificación correcta
html_content = response.text

# Parsear el contenido HTML con BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Mostrar los primeros 1000 caracteres para verificar que la página se haya descargado correctamente
print(html_content[:1000])


<!DOCTYPE html>
<html class='ltr' dir='ltr' lang='es' xmlns='http://www.w3.org/1999/xhtml' xmlns:b='http://www.google.com/2005/gml/b' xmlns:data='http://www.google.com/2005/gml/data' xmlns:expr='http://www.google.com/2005/gml/expr'>
<head>
<link href='https://www.bloghemia.com/2019/05/noam-chomsky-michel-foucault-debate.html' rel='canonical'/>
<script async='async' data-ad-client='ca-pub-3126310743890539' src='https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js'></script>
<title>Justicia vs Poder |  Transcripción completa al español</title>
<!-- Descripción optimizada -->
<!-- Descripción dinámica para entradas y otras páginas -->
<meta content='Transcripción del Debate entre Noam Chomsky y Michel Foucault, celebrado en la Universidad de Amsterdam ' name='description'/>
<meta content='Bloghemia' name='author'/>
<meta content='https://www.bloghemia.com/' name='identifier-url'/>
<meta content='index, follow' name='robots'/>
<meta content='Bloghemia' name='Copyright'/>
<meta co


#### Extraer su encabezado y armar un dataframe agrupando sus contenidos por tipo de etiqueta (hint: groupby), excepto por lo que esté relacionado con la inline stylesheet pues esto último se escribirá a un archivo CSS.


In [78]:
# Extraer el encabezado
head = soup.head

# Crear un diccionario para almacenar el contenido agrupado por etiqueta
header_tags = {}

for tag in head.find_all():
    tag_name = tag.name
    if tag_name not in header_tags:
        header_tags[tag_name] = []
    header_tags[tag_name].append(str(tag))

# Convertir en DataFrame
df_header = pd.DataFrame([(key, value) for key, value in header_tags.items()], columns=["Etiqueta", "Contenido"])

# Aplicar groupby para ver cuántos elementos hay por tipo de etiqueta
df_grouped = df_header.groupby("Etiqueta").count()

#### Eliminar todos los inline scripts y otras etiquetas que estén relacionados con anuncios y todo tipo de publicidad; hacer esto tanto para el encabezado como para el cuerpo del archivo HTML.


In [81]:
# Eliminar scripts en línea
for script in soup.find_all('script'):
    script.decompose()

# Lista de palabras clave relacionadas con anuncios
ad_keywords = ['ad', 'advertisement', 'banner', 'promo', 'googleads', 'doubleclick', 'googlesyndication']

# Eliminar etiquetas con atributos que contengan palabras clave de publicidad
for ad_keyword in ad_keywords:
    for element in soup.find_all(attrs={'class': lambda x: x and ad_keyword in x}):
        element.decompose()
    for element in soup.find_all(attrs={'id': lambda x: x and ad_keyword in x}):
        element.decompose()
    for element in soup.find_all('meta', attrs={'content': lambda x: x and ad_keyword in x}):
        element.decompose()
    for element in soup.find_all('noscript'):
        if any(ad_keyword in str(element) for ad_keyword in ad_keywords):
            element.decompose()
    for element in soup.find_all('link', attrs={'href': lambda x: x and ad_keyword in x}):
        element.decompose()


Para este paso en específico tuve que dedicarle más tiempo ya que me seguían quedando multiples objetos de publicidad, por lo que tuve que hacer una busqueda más exhaustiva

#### Del dataframe del paso 1 obtenga todos los metadatos y escríbalos a un archivo Excel llamado metadata.xls


In [80]:
# Guardar los metadatos en un archivo Excel
metadata_path = "metadata.xlsx"
df_grouped.to_excel(metadata_path, index=False)

print(f"Metadatos guardados en {metadata_path}")
print(df_grouped.head)

Metadatos guardados en metadata.xlsx
<bound method NDFrame.head of           Contenido
Etiqueta           
link              1
meta              1
noscript          1
script            1
style             1
title             1>


#### Con el contenido del cuerpo se van a generar dos archivos:

#### A). Un archivo HTML “mínimo” donde solamente estarán (en orden) las intervenciones de las 3 personas que intervinieron en el debate. Para el encabezado de este HTML puede usar el título del documento fuente.


In [82]:
# Eliminar otros elementos irrelevantes (iframes, estilos en línea, etc.)
for element in soup(["iframe", "ins", "noscript", "style"]):
    element.decompose()

# Extraer el contenido limpio
clean_html = str(soup)

# Guardar la versión limpia en un archivo
html_clean_path = "clean_page.html"
with open(html_clean_path, "w", encoding="utf-8") as file:
    file.write(clean_html)

Para este archivo, realmente sigo teniendo muchos elementos irrelevantes, sin embargo, encontré muchos problemas al querer limpiar más, llegando incluso a borrar todo y no pude refinar la limpieza para quedarme solo con lo necesario, por lo que por el momento lo dejaré así

#### B). Un archivo JSON producto de un dataframe donde se agrupen las intervenciones de cada interlocutor.

In [84]:
# Extraer todas las etiquetas <span> con intervenciones
span_tags = soup.find_all("span")

# Inicializar variables
debate_data = []
current_speaker = None

# Analizar cada etiqueta span
for span in span_tags:
    text = span.get_text(strip=True)

    # Detectar cambios de interlocutor
    if text.startswith(("ELDERS:", "CHOMSKY:", "FOUCAULT:")):
        current_speaker = text.split(":")[0]
        text = text[len(current_speaker) + 1:].strip()  # Quitar el nombre del hablante

    # Si seguimos con el mismo hablante, agregar la intervención
    if current_speaker:
        debate_data.append({"Interlocutor": current_speaker, "Intervención": text})

# Convertir en DataFrame
df_debate = pd.DataFrame(debate_data)

# Agrupar las intervenciones por interlocutor
df_grouped_debate = df_debate.groupby("Interlocutor")["Intervención"].apply(lambda x: " ".join(x)).reset_index()

# Guardar en JSON
json_path = "debate.json"
df_grouped_debate.to_json(json_path, orient="records", indent=4, force_ascii=False)



Para este paso fue relativamente sencillo ya que solo tuve que extraer los elementos span y hacer la busqueda en base al inicio de los parrafos que mencionan a la persona al inicio de cada intervención.

## Práctica 2

#### Ahora use archivos html (uno o más) de alguna de las fuentes de datos que utilizará para su proyecto.
#### Primero construya el parse tree de una página y recorralo usando los métodos disponibles (i.e., contents, children, descendants, parent, parents, next_sibling, and previous_sibling). Envíe a un archivo de texto (o bien con formateo html) el resultado de este recorrido.


In [92]:
# Cargar el archivo HTML
file_path = "Requiem - Halopedia, the Halo wiki.html"  # Ajusta la ruta según sea necesario
with open(file_path, "r", encoding="utf-8") as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, "html.parser")

parse_tree_output = []

# contents y children
parse_tree_output.append("\n### Body contents ###")
for child in soup.body.contents[:5]:  # Mostramos solo los primeros 5
    parse_tree_output.append(str(child)[:200] + "...")

parse_tree_output.append("\n### Body children ###")
for child in soup.body.children:
    parse_tree_output.append(str(child)[:200] + "...")
    break  # Solo mostramos el primero para evitar contenido largo

parse_tree_output.append("\n### Body descendants ###")
for i, descendant in enumerate(soup.body.descendants):
    if i > 5:  # Limitar a los primeros 5
        break
    parse_tree_output.append(str(descendant)[:200] + "...")

# parent y parents de un elemento específico (el título h1)
title_element = soup.find("h1")
parse_tree_output.append("\n### Parent of title ###")
parse_tree_output.append(str(title_element.parent)[:200] + "...")

# next_sibling y previous_sibling de un párrafo
first_paragraph = soup.find("p")
parse_tree_output.append("\n### Next sibling of first paragraph ###")
parse_tree_output.append(str(first_paragraph.next_sibling)[:200] + "...")

parse_tree_output.append("\n### Previous sibling of first paragraph ###")
parse_tree_output.append(str(first_paragraph.previous_sibling)[:200] + "...")

# Guardar el resultado del recorrido en un archivo de texto
parse_tree_path = "parse_tree_traversal.txt"
with open(parse_tree_path, "w", encoding="utf-8") as file:
    file.write("\n".join(parse_tree_output))

print("Resultado del recorrido guardado en", parse_tree_path)

Resultado del recorrido guardado en parse_tree_traversal.txt


Para este paso, se dependió principalmente de ciclos y de las funciones de soup para hacer append a los distintos objetos como contents y children (soup.body.contents), realmente el uso de esta función y de soup.find es lo unico que vale la pena mencionar del script ya que lo demás ya se había practicado anteriormente.


#### Con el parse tree haga por lo menos un ejemplo con cada método de búsqueda disponible (i.e., find_all, find, select, and select_one). Haga al menos una búsqueda utilizando expresiones regulares.



In [93]:
# find_all para encontrar todos los enlaces
all_links = soup.find_all("a")
print(f"Total links found: {len(all_links)}")

# find para encontrar el primer enlace
first_link = soup.find("a")
print(f"First link: {first_link}")

# select para buscar con CSS selector
infobox = soup.select("table.infobox")
print(f"Infobox count: {len(infobox)}")

# select_one para encontrar el título principal
page_title = soup.select_one("h1")
print(f"Page title: {page_title.text}")

# Búsqueda con expresiones regulares para encontrar textos con "Requiem"
regex_search = soup.find_all(string=re.compile("Requiem"))
print(f"Occurrences of 'Requiem': {len(regex_search)}")

Total links found: 1014
First link: <a class="mw-skin-nimbus-button positive-button" href="https://www.halopedia.org/Special:CreateAccount" rel="nofollow"><span>Sign up</span></a>
Infobox count: 1
Page title: Requiem
Occurrences of 'Requiem': 114


Para este paso, se uso soup.find, por lo que el proceso fue muy simple y solo es cuestión de especificar el tipo de busqueda a realizar.

#### Ahora utilice los métodos disponibles para modificarlo, incluyendo agregar (i.e., append, insert, new_tag) y remover (i.e., decompose, extract).


In [102]:
# Agregar un nuevo párrafo
new_paragraph = soup.new_tag("p")
new_paragraph.string = "This is a newly inserted paragraph for testing purposes."
soup.body.insert(0, new_paragraph)

# Remover un elemento (por ejemplo, el primer enlace)
if first_link:
    first_link.decompose()
    print("First link removed.")

# Insertar un nuevo encabezado
new_heading = soup.new_tag("h2")
new_heading.string = "New Section Added"
soup.body.insert(1, new_heading)

# Agregar una lista desordenada
new_list = soup.new_tag("ul")
for item_text in ["Item 1", "Item 2", "Item 3"]:
    new_item = soup.new_tag("li")
    new_item.string = item_text
    new_list.append(new_item)
soup.body.insert(2, new_list)

# Extraer y mover un párrafo al final del cuerpo
last_paragraph = soup.find("p")
if last_paragraph:
    extracted_paragraph = last_paragraph.extract()
    soup.body.append(extracted_paragraph)

First link removed.


Para la modificación, se usó principalmente soup.body.insert ya que nos permite agregar diferentes tipos de objetos, para eliminar se utilizó .extract en el objetó recibido con soup.find

In [96]:
modified_html_path = "modified_html.html"
with open(modified_html_path, "w", encoding="utf-8") as file:
    file.write(soup.prettify())

print(f"Modified HTML saved to {modified_html_path}")

Modified HTML saved to modified_html.html


A pesar de que no puedo pegar imagenes aquí, se puede observar cómo algunos elementos se perdieron con las modificaciones ya que corrí el código varias veces y también se pueden ver los elementos agregados, es interesante ver los cambios generados por las modificaciones ya que se perdieron un par de elementos clave que hacen que la página cambie mucho, y se puede observar claramente la simpleza de los elementos agregados.

## Práctica 3
##### Usando el archivo https://media.geeksforgeeks.org/wp-content/uploads/employees.csv cargue todo en un dataframe y a partir de ahí realice las siguientes operaciones:


In [97]:
# Cargar el archivo CSV
file_path = "employees.csv"
df = pd.read_csv(file_path)

# Mostrar las primeras filas del DataFrame
print(df.head())

  First Name  Gender Start Date Last Login Time  Salary  Bonus %  \
0    Douglas    Male   8/6/1993        12:42 PM   97308    6.945   
1     Thomas    Male  3/31/1996         6:53 AM   61933    4.170   
2      Maria  Female  4/23/1993        11:17 AM  130590   11.858   
3      Jerry    Male   3/4/2005         1:00 PM  138705    9.340   
4      Larry    Male  1/24/1998         4:47 PM  101004    1.389   

  Senior Management             Team  
0              True        Marketing  
1              True              NaN  
2             False          Finance  
3              True          Finance  
4              True  Client Services  


#### Despliegue solo los registros donde está faltante la información de la columna Gender (hint: isnull).


In [98]:
# Filtrar registros donde la columna 'Gender' es nula
missing_gender_df = df[df["Gender"].isnull()]

# Mostrar los registros con valores faltantes en 'Gender'
print(missing_gender_df)

    First Name Gender  Start Date Last Login Time  Salary  Bonus %  \
20        Lois    NaN   4/22/1995         7:18 PM   64714    4.934   
22      Joshua    NaN    3/8/2012         1:58 AM   90816   18.816   
27       Scott    NaN   7/11/1991         6:58 PM  122367    5.218   
31       Joyce    NaN   2/20/2005         2:40 PM   88657   12.752   
41   Christine    NaN   6/28/2015         1:08 AM   66582   11.308   
..         ...    ...         ...             ...     ...      ...   
961    Antonio    NaN   6/18/1989         9:37 PM  103050    3.050   
972     Victor    NaN   7/28/2006         2:49 PM   76381   11.159   
985    Stephen    NaN   7/10/1983         8:10 PM   85668    1.909   
989     Justin    NaN   2/10/1991         4:58 PM   38344    3.794   
995      Henry    NaN  11/23/2014         6:09 AM  132483   16.655   

    Senior Management                  Team  
20               True                 Legal  
22               True       Client Services  
27              False

#### Ahora haga lo contrario, es decir, muestre solo los registros donde no faltan la información de la columna Gender (hint: notnull).

In [99]:
# Filtrar registros donde la columna 'Gender' no es nula
not_missing_gender_df = df[df["Gender"].notnull()]

# Mostrar los registros con valores no nulos en 'Gender'
print(not_missing_gender_df)

    First Name  Gender Start Date Last Login Time  Salary  Bonus %  \
0      Douglas    Male   8/6/1993        12:42 PM   97308    6.945   
1       Thomas    Male  3/31/1996         6:53 AM   61933    4.170   
2        Maria  Female  4/23/1993        11:17 AM  130590   11.858   
3        Jerry    Male   3/4/2005         1:00 PM  138705    9.340   
4        Larry    Male  1/24/1998         4:47 PM  101004    1.389   
..         ...     ...        ...             ...     ...      ...   
994     George    Male  6/21/2013         5:47 PM   98874    4.479   
996    Phillip    Male  1/31/1984         6:30 AM   42392   19.675   
997    Russell    Male  5/20/2013        12:39 PM   96914    1.421   
998      Larry    Male  4/20/2013         4:45 PM   60500   11.985   
999     Albert    Male  5/15/2012         6:24 PM  129949   10.169   

    Senior Management                  Team  
0                True             Marketing  
1                True                   NaN  
2               False

#### En todos los registros donde falta algún valor (en cualquier campo), remplace dicho valor (Hint: replace) con algún otro que resulte más apropiado de manejar (e.g., en Gender puede poner UNSPECIFIED, en Salary un 00.00, etc.)


In [100]:
# Reemplazar valores nulos con valores adecuados
df_filled = df.fillna({
    "Gender": "UNSPECIFIED",         # Reemplazar valores nulos en 'Gender' con 'UNSPECIFIED'
    "Start Date": "UNKNOWN",         # Reemplazar fechas faltantes con 'UNKNOWN'
    "Last Login Time": "UNKNOWN",    # Reemplazar tiempos faltantes con 'UNKNOWN'
    "Salary": 0.00,                  # Reemplazar salarios faltantes con 0.00
    "Bonus %": 0.00,                 # Reemplazar valores nulos en 'Bonus %' con 0.00
    "Senior Management": False,      # Asumir 'False' si falta información de senioridad
    "Team": "NO TEAM"                # Reemplazar valores nulos en 'Team' con 'NO TEAM'
})

# Mostrar los primeros registros después de la limpieza
print(df_filled.head())

  First Name  Gender Start Date Last Login Time  Salary  Bonus %  \
0    Douglas    Male   8/6/1993        12:42 PM   97308    6.945   
1     Thomas    Male  3/31/1996         6:53 AM   61933    4.170   
2      Maria  Female  4/23/1993        11:17 AM  130590   11.858   
3      Jerry    Male   3/4/2005         1:00 PM  138705    9.340   
4      Larry    Male  1/24/1998         4:47 PM  101004    1.389   

   Senior Management             Team  
0               True        Marketing  
1               True          NO TEAM  
2              False          Finance  
3               True          Finance  
4               True  Client Services  


C:\Users\lizar\AppData\Local\Temp\ipykernel_364\2682820789.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filled = df.fillna({


#### Muestre el número total de registros, el número donde falta al menos un valor en cualquier campo y el número de registros sin valores faltantes. Solo muestre números, no los registros mismos.


In [101]:
# Número total de registros
total_records = len(df)

# Número de registros con al menos un valor faltante
missing_records = df.isnull().any(axis=1).sum()

# Número de registros sin valores faltantes
complete_records = total_records - missing_records

# Mostrar los resultados
print(total_records)
print(missing_records)
print(complete_records)


1000
236
764


Para esta uiltima práctica, fue la más sencilla principalmente porque pandas ya tiene funciones built in como isnull para hacer este tipo de revisiones, lo que facilita mucho este tipo de manipulación de datos.

##### A manera de conclusión, fue interesante trabajar con estas herramientas para ya una manipulación más material de lo que haremos como proyecto, en especifico la primera práctica me presentó un par de retos que fueron solucionables en su mayoría, mientras que las otras dos fueron interesantes para el uso de las distintas herramientas que nos ofrecen las librerías.
